## OpenML in research, iris

It covers downloading datasets, tasks, how to use scikit-learn to build classifiers, and upload the results to the server.


Initialization and login. This assumes you have a .openml dir in your homedir with a subdir for caches and a file with your API key. You find your API key in your account settings on openml.org.

In [1]:
!pip freeze

backports.ssl-match-hostname==3.4.0.2
bokeh==0.10.0
climate==0.4.5
decorator==4.0.4
downhill==0.3.1
Flask==0.10.1
funcsigs==0.4
ipykernel==4.1.1
ipython==4.0.0
ipython-genutils==0.1.0
itsdangerous==0.24
Jinja2==2.8
jsonschema==2.4.0
jupyter-client==4.1.1
jupyter-core==4.0.6
liac-arff==2.1.1.dev0
MarkupSafe==0.23
matplotlib==1.4.3
mistune==0.7.1
mock==1.3.0
mpld3==0.2
nbconvert==4.0.0
nbformat==4.0.1
neurolab==0.3.5
nose==1.3.7
notebook==4.0.6
numexpr==2.4.4
numpy==1.10.1
openml==0.2.1
openpyxl==1.8.6
pandas==0.17.0
path.py==0.0.0
pbr==1.8.1
pexpect==3.3
pickleshare==0.5
plac==0.9.1
plotly==1.8.6
ptyprocess==0.5
PyBrain==0.3
Pygments==2.0.2
pyparsing==2.0.3
python-dateutil==2.4.2
pytz==2015.7
PyYAML==3.11
pyzmq==14.7.0
rep==0.6.5
requests==2.8.1
root-numpy==4.3.0.dev0
rootpy==0.8.0.dev0
runipy==0.1.3
scikit-learn==0.16.1
scipy==0.16.0
simplegeneric==0.8.1
six==1.10.0
tables==3.2.2
terminado==0.5
theanets==0.6.2
Theano==0.7.0
tornado==4.2.1
traitlets==4.0.0
Werkzeug==0.10.4
wheel==0.26.0

In [2]:
%%bash

python --version
which pip

/root/miniconda/envs/rep_py2/bin/pip


Python 2.7.10 :: Continuum Analytics, Inc.


In [3]:
from sklearn import preprocessing, ensemble
from openml.apiconnector import APIConnector
import numpy as np
import pandas as pd
import os
import openml
import xmltodict
from IPython.core.display import display, HTML


openml_dir = os.path.join(os.path.expanduser("~"), ".openml")
if not os.path.exists(openml_dir): os.makedirs(openml_dir)
cache_dir = os.path.join(openml_dir, "cache")
key = "ef035bbd1ca47af239c384aea9124ec5"

ImportError: No module named apiconnector

In [4]:
con = APIConnector(cache_directory=cache_dir, apikey=key)

NameError: name 'APIConnector' is not defined

In [9]:
from sklearn import preprocessing, ensemble
from openml.apiconnector import APIConnector
import openml
import numpy as np
import pandas as pd
import os

ImportError: No module named apiconnector

List all datasets on OpenML

In [14]:
datasets = openml.datasets.get_dataset_list(con)

datasets_df = pd.DataFrame(datasets)
datasets_df.head()


AttributeError: 'module' object has no attribute 'get_dataset_list'

Download a specific dataset. This is done based on the dataset ID (called 'did' in the table above).

In [ ]:
from pprint import pprint
import arff

N = 61
print("Downloading dataset %d." % N)
dataset = openml.datasets.download_dataset(con, N)

print("This is dataset '%s', the target feature is called '%s'" % (dataset.name, dataset.default_target_attribute))
print("More info, including the location off the .arff file on disk:")
pprint(vars(dataset))


In [ ]:
X, y = dataset.get_dataset(target=dataset.default_target_attribute)

In [ ]:
features = openml.datasets.download_dataset_features(con, N)

In [ ]:
features

In [ ]:
iris = pd.DataFrame(X)
iris['class'] = y
print(iris[:10])

Training a scikit-learn model with the data

In [ ]:
clf = ensemble.RandomForestClassifier()
clf.fit(X, y)

You can also ask with features are categorical to do your own encoding 

In [ ]:
task_list = openml.tasks.get_task_list(con)

In [ ]:
print(task_list[0], len(task_list))

In [ ]:
tasks = pd.DataFrame(task_list)
tasks[['tid','did','task_type','NumberOfInstances','NumberOfFeatures','NumberOfClasses']].head(11)

Download a single OpenML task (id=10), create a scikit-learn classifier (RandomForest), and run it on the task

In [ ]:
T = 10107
task = openml.tasks.download_task(con, T)
print(task)

clf = ensemble.RandomForestClassifier()

In [ ]:
run = openml.runs.openml_run(con, task, clf)
print("RandomForest has run on the task.")

Upload the run to the OpenML server

In [ ]:
return_code, response = run.publish(con)

In [ ]:
if(return_code == 200):
    response_dict = xmltodict.parse(response)
    run_id = response_dict['oml:upload_run']['oml:run_id']
    print("Uploaded run with id %s" % (run_id))
    display(HTML("<a href='http://www.openml.org/r/{0}' target=_blank>http://www.openml.org/r/{0}</a>".format(run_id)))
